In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import torch
import torch.nn.functional as F
from datasets import load_dataset, Dataset, DatasetDict
import huggingface_hub
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    EarlyStoppingCallback,
    DataCollatorForSeq2Seq
)

from trl.core import LengthSampler
from trl import (
    PPOTrainer,
    PPOConfig,
    AutoModelForCausalLMWithValueHead,
    create_reference_model,
    DPOConfig,
    DPOTrainer,
)
from peft import (
    get_peft_model,
    AutoPeftModel,
    AutoPeftModelForCausalLM,
    PeftModel,
    LoraConfig,
    LoftQConfig,
    TaskType,
)
from trl import SFTTrainer
from unsloth import is_bfloat16_supported
from unsloth import FastLanguageModel
from unsloth.chat_templates import (
    get_chat_template,
    train_on_responses_only,
    standardize_sharegpt,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = "1"

In [3]:
model_name = 'HoJL/qa_v3'
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    load_in_4bit=True,
    dtype=None,
    max_seq_length=2048
)
model = model.float()

==((====))==  Unsloth 2024.12.9: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA A100 80GB PCIe. Max memory: 79.254 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.12.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [4]:
dataset = load_dataset('HoJL/context_qa_set', split='train')

In [5]:
data = dataset.train_test_split(test_size=0.1)
train_test = data['train'].train_test_split(test_size=0.1)
dataset = DatasetDict({
    'train': train_test['train'],
    'valid': data['test'],
    'test': train_test['test']
})

In [6]:
from transformers import StoppingCriteria, StoppingCriteriaList


class StopOnToken(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id  # 정지 토큰 ID를 초기화합니다.

    def __call__(self, input_ids, scores, **kwargs):
        return (
            self.stop_token_id in input_ids[0]
        )  # 입력된 ID 중 정지 토큰 ID가 있으면 정지합니다.


# end_token을 설정
stop_token = "<|end_of_text|>"  # end_token으로 사용할 토큰을 설정합니다.
stop_token_id = tokenizer.encode(stop_token, add_special_tokens=False)[
    0
]

# Stopping criteria 설정
stopping_criteria = StoppingCriteriaList(
    [StopOnToken(stop_token_id)]
)  # 정지 조건을 설정합니다.

In [7]:
inference_prompt = """
너는 내용을 보고 질문에 맞는 답을 해주는 역할이야.

###내용:
{}
###질문:
{}
###답:
{}
"""

In [8]:
texts = [
        inference_prompt.format(
            row.context,
            row.question,  # 지시사항
            "",  # 출력 - 생성을 위해 이 부분을 비워둡니다!
        ) for row in dataset['test'].to_pandas().itertuples()

]

In [9]:
context = dataset['test'][3]['context']
question = dataset['test'][3]['question']

In [10]:
text = [inference_prompt.format(
            context,
            question,  # 지시사항
            "",  # 출력 - 생성을 위해 이 부분을 비워둡니다!
        )]

In [36]:
from transformers import TextStreamer
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"
model.config.pad_token_id = model.config.eos_token_id
# FastLanguageModel을 이용하여 추론 속도를 2배 빠르게 설정합니다.
print(tokenizer.padding_side)
FastLanguageModel.for_inference(model)
inputs = tokenizer(
    texts[4],
    return_tensors="pt",
    padding=True
).to('cuda')

left


In [37]:
for key in inputs:
    if isinstance(inputs[key], torch.Tensor):
        inputs[key] = torch.nan_to_num(inputs[key], nan=0.0, posinf=1.0, neginf=-1.0)

In [38]:
len(inputs.get('input_ids')[0])

350

In [39]:
text_streamer = TextStreamer(tokenizer)

with torch.no_grad():
    output = model.generate(
        **inputs,
        # streamer=text_streamer,
        max_new_tokens=2048,  # 최대 생성 토큰 수를 설정합니다.
        # stopping_criteria=stopping_criteria,  # 생성을 멈출 기준을 설정합니다.
    )

In [40]:
result = tokenizer.batch_decode(output, skip_special_tokens=True)

In [41]:
result[0]

'\n너는 내용을 보고 질문에 맞는 답을 해주는 역할이야.\n\n###내용:\npassage 1: 문의하기. Dove® 웹사이트를 방문해 주셔서 감사합니다. 실시간 상담원과 통화하시려면 월요일부터 금요일까지 동부 시간 기준 오전 8:30부터 오후 6:00까지 1-800-761-DOVE (3683)로 전화해 주십시오. 의료 또는 제품 안전 비상 상황인 경우, 1-800-297-7421로 전화해 주십시오.\n\npassage 2: Dove® 웹사이트를 방문해 주셔서 감사합니다. 실시간 상담원과 통화하시려면 월요일부터 금요일까지 동부 시간 기준 오전 8:30부터 오후 6:00까지 1-800-761-DOVE (3683)로 전화해 주십시오. 의료 또는 제품 안전 비상 상황인 경우, 1-800-297-7421로 전화해 주십시오. 이 번호는 하루 24시간, 주 7일 이용 가능합니다.\n\npassage 3: 공유하기. Dove는 아름다움이란 최상의 상태를 느끼고 보이는 것이 적절한 관리의 결과라고 믿습니다. Dove는 항상 피부나 머리카락의 상태를 눈에 띄게 개선하고, 관리의 즐거운 경험을 제공하는 제품을 제공하는 것을 목표로 합니다. 왜냐하면 아름답게 보이고 느낄 때 더 행복해지기 때문입니다.\n###질문:\n도브 비누 본사는 어디에 있나요?\n###답:\n\n주어진 문구를 바탕으로, 도브 비누 본사는 다음과 같은 정보를 제공합니다:\n\n* 도브 비누 본사는 도브 웹사이트를 방문해 주셔서 감사합니다 (문구 1). 도브 웹사이트를 방문하여 도브 비누의 제품과 서비스에 대한 정보를 찾으세요.\n* 도브 비누 실시간 상담원은 월요일부터 금요일까지 동부 시간 기준 오전 8:30부터 오후 6:00까지 1-800-761-DOVE(3683)로 전화하실 수 있습니다 (문구 1). 도브 비누 실시간 상담원은 도브 비누의 제품과 서비스에 대한 질문이나 문제를 해결하실 수 있습니다.\n* 도브 비누의 제품 안전 비상 상황은 1-800-297-7421로 전화하실 수 있습니다 (문구 2). 